# Dataset Preparation
* Dataset is IMDB Movie Reviews
* Data can be found at https://www.kaggle.com/code/badnick/spoilerdetector

## Glove Embeddings
* Glove embeddings file can be downloaded at the following link: https://nlp.stanford.edu/projects/glove/
* Download the file called glove.6B.zip
* Expand the downloaded zip and use the file called "glove.6B.300d.txt"

In [18]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [6]:
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('words')

In [8]:
# Will take a little over a minute to load
full_df = pd.read_csv("../cleaned_reviews_summaries.zip")

/Users/anuraag_govindarajan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Generate Vocab and Embeddings

In [9]:
from sklearn.model_selection import train_test_split
import torch

In [10]:
full_df.head()

,Unnamed: 0,review_date,movie_id,user_id,is_spoiler,review_text,rating_x,review_summary,plot_summary,duration,genre,rating_y,release_date,plot_synopsis,cleaned_summaries,cleaned_reviews
0,0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10.0,A classic piece of unforgettable film-making.,Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,oscar year shawshank redemption written direct...
1,1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10.0,Simply amazing. The best film of the 90's.,Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,shawshank redemption without doubt one brillia...
2,2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8.0,The best story ever told on film,Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,believe film best story ever told film tell wh...
3,3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10.0,Busy dying or busy living?,Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,yes spoilers herethis film emotional impact fi...
4,4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8.0,"Great story, wondrously told and acted",Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,heart extraordinary movie brilliant indelible ...


In [11]:
df_mini = full_df.sample(500) # Small Size to test out functions

In [12]:
df_mini.head()

,Unnamed: 0,review_date,movie_id,user_id,is_spoiler,review_text,rating_x,review_summary,plot_summary,duration,genre,rating_y,release_date,plot_synopsis,cleaned_summaries,cleaned_reviews
562257,562257,31 August 2005,tt0172493,ur0989035,False,"After being touched by the sensitive film ""Hea...",7.0,Director James Mangold hits a slight interrupt...,Unable to cope with reality and the difficulty...,2h 7min,"['Biography', 'Drama']",7.4,2000-01-14,"""Maybe I really was crazy. Maybe it was the 60...",unable cope reality difficulty comes year old ...,touched sensitive film heavy could wait see wo...
536258,536258,10 January 2005,tt0119396,ur2082018,False,Quentin Tarantino follow up to the hysterical ...,8.0,Tarantino grows up,The middle-aged stewardess Jackie Brown smuggl...,2h 34min,"['Crime', 'Drama', 'Thriller']",7.5,1997-12-25,"Los Angeles,1995. Jackie Brown (Pam Grier) is ...",middleaged stewardess jackie brown smuggles mo...,quentin tarantino follow hysterical response g...
92728,92728,23 July 2017,tt2119532,ur55428015,False,This movie reached my heart and soul which is ...,10.0,Masterpiece,"The true story of Desmond T. Doss, the conscie...",2h 19min,"['Biography', 'Drama', 'History']",8.2,2016-11-04,"The film starts with very VERY graphic, viscer...",true story desmond doss conscientious objector...,movie reached heart soul extraordinary ability...
454128,454128,9 April 2010,tt0441773,ur22879123,True,When I realized that I don't like so much cart...,8.0,Attracting funny and wise,"It's the story about a lazy, irreverent slacke...",1h 32min,"['Animation', 'Action', 'Adventure']",7.6,2008-06-06,"""Legend tells of a legendary warrior whose kun...",story lazy irreverent slacker panda named po b...,realized like much cartoons watch maybe one se...
424101,424101,7 June 2006,tt0452594,ur10948076,False,I have to admit I went in to the theater to wa...,7.0,Good examination or a breakup,"In Chicago, the art dealer Brooke Meyers feels...",1h 46min,"['Comedy', 'Drama', 'Romance']",5.8,2006-06-02,Gary Grobowski (Vince Vaughn) and Brooke Meyer...,chicago art dealer brooke meyers feels appreci...,admit went theater watch breakup without high ...


In [13]:
EMB_DIM = 300
vocab = set()
word_to_id = dict()
id_to_word = dict()
word_to_count = dict()
vocab_size = 1

In [14]:
max_len = 30 # placeholder value

In [15]:
# cols = ['cleaned_reviews', 'cleaned_summaries'] # Use if you want vocab to include plot summaries
cols = ['cleaned_reviews'] # Use if you want vocab to include just review text

### Generate Vocabulary

In [17]:
def generate_vocab(cols):
    
    global vocab_size, vocab
    global word_to_id, id_to_word, word_to_count
    global full_df
    global df_mini
    cleaned_reviews_ids = []
    cleaned_summaries_ids = []
    
    for index, row in full_df.iterrows():
        
        for c in cols:
            
            s2n = []
            split = row[c].split()
            
            for word in split:
                
                if word not in vocab:
                    vocab.add(word)
                    word_to_id[word] = vocab_size
                    # word_to_count[word] = 1
                    s2n.append(vocab_size)
                    id_to_word[vocab_size] = word
                    vocab_size += 1
                    
                else:
                    # word_to_count[word] += 1
                    s2n.append(word_to_id[word])
                    
            
            full_df.at[index, c] = s2n
            

In [19]:
# Takes about 100 seconds with just cleaned reviews column, will take longer if you include cleaned summaries
start = time.time()
generate_vocab(cols)
print(time.time() - start)

100.90569996833801


In [20]:
print(len(vocab))

1085160


In [25]:
full_df.head()

,Unnamed: 0,review_date,movie_id,user_id,is_spoiler,review_text,rating_x,review_summary,plot_summary,duration,genre,rating_y,release_date,plot_synopsis,cleaned_summaries,cleaned_reviews
0,0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10.0,A classic piece of unforgettable film-making.,Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 3, 4, 12, ..."
1,1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10.0,Simply amazing. The best film of the 90's.,Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,"[3, 4, 321, 322, 187, 323, 190, 324, 309, 325,..."
2,2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8.0,The best story ever told on film,Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,"[385, 35, 21, 94, 324, 386, 35, 387, 388, 81, ..."
3,3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10.0,Busy dying or busy living?,Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,"[466, 467, 468, 35, 469, 470, 471, 472, 473, 4..."
4,4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8.0,"Great story, wondrously told and acted",Chronicles the experiences of a formerly succe...,2h 22min,"['Crime', 'Drama']",9.3,1994-10-14,"In 1947, Andy Dufresne (Tim Robbins), a banker...",chronicles experiences formerly successful ban...,"[338, 638, 86, 323, 639, 640, 165, 166, 164, 4..."


In [26]:
X_train = []
y_train = []
X_test = []
y_test = []
X_val = []
y_val = []

### Split data into training, testing, and validation

In [33]:
full_df['is_spoiler'] = full_df['is_spoiler'].fillna(0)
full_df['is_spoiler'].isna().sum()

0

In [34]:
def split_data(cols):
    
    global X_train, y_train
    global X_test, y_test
    global X_val, y_val
    
    train_ratio = 0.8 # Split into training and validation
    test_ratio = 0.85 # Split into training and testing
    
    X = list(full_df['cleaned_reviews'])
    y = list(full_df['is_spoiler'].astype(int))
    
    X_rem, X_test, y_rem, y_test = train_test_split(X, y, train_size=test_ratio)
    X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=train_ratio)
    
    # X_train = [X_train[c] for c in cols]
    # X_val = [X_val[c] for c in cols]
    # X_test = [X_test[c] for c in cols]
    
    # y_train = y_train.values
    # y_test = y_test.values
    # y_val = y_val.values

In [35]:
start = time.time()
split_data(cols)
print(time.time() - start)

0.7105481624603271


In [36]:
print(len(X_train), len(y_train), len(X_test), len(y_test))

390261 390261 86088 86088


In [37]:
def pad_sequences(X_list):
    
    X_padded = torch.nn.utils.rnn.pad_sequence([torch.as_tensor(l) for l in X_list]).type(torch.LongTensor) # padding the sequences with 0
    X_mask   = torch.nn.utils.rnn.pad_sequence([torch.as_tensor([1.0] * len(l)) for l in X_list]).type(torch.FloatTensor)
    
    return X_padded, X_mask

In [39]:
# Takes about 20 seconds to run
start = time.time()
X_train = pad_sequences(X_train)[0]
print(time.time() - start)

16.928745985031128


### Get Glove embeddings dictionary

In [40]:
# Read GloVe embeddings.
def read_GloVe(filename):
    
    embeddings = {}
    for line in open(filename).readlines():
        #print(line)
        fields = line.strip().split(" ")
        word = fields[0]
        embeddings[word] = [float(x) for x in fields[1:]]
        
    return embeddings

In [42]:
# Takes about 40 seconds to run
start = time.time()
GloVe = read_GloVe("../glove/glove.6B.300d.txt")

print(GloVe["the"])
print("dimension of glove embedding:", len(GloVe["the"]))
print()
print(time.time() - start)

[0.04656, 0.21318, -0.0074364, -0.45854, -0.035639, 0.23643, -0.28836, 0.21521, -0.13486, -1.6413, -0.26091, 0.032434, 0.056621, -0.043296, -0.021672, 0.22476, -0.075129, -0.067018, -0.14247, 0.038825, -0.18951, 0.29977, 0.39305, 0.17887, -0.17343, -0.21178, 0.23617, -0.063681, -0.42318, -0.11661, 0.093754, 0.17296, -0.33073, 0.49112, -0.68995, -0.092462, 0.24742, -0.17991, 0.097908, 0.083118, 0.15299, -0.27276, -0.038934, 0.54453, 0.53737, 0.29105, -0.0073514, 0.04788, -0.4076, -0.026759, 0.17919, 0.010977, -0.10963, -0.26395, 0.07399, 0.26236, -0.1508, 0.34623, 0.25758, 0.11971, -0.037135, -0.071593, 0.43898, -0.040764, 0.016425, -0.4464, 0.17197, 0.046246, 0.058639, 0.041499, 0.53948, 0.52495, 0.11361, -0.048315, -0.36385, 0.18704, 0.092761, -0.11129, -0.42085, 0.13992, -0.39338, -0.067945, 0.12188, 0.16707, 0.075169, -0.015529, -0.19499, 0.19638, 0.053194, 0.2517, -0.34845, -0.10638, -0.34692, -0.19024, -0.2004, 0.12154, -0.29208, 0.023353, -0.11618, -0.35768, 0.062304, 0.35884, 0.

### Generate pretrained embeddings for our vocabulary

In [43]:
def init_glove(GloVe):
    
    global EMB_DIM
    
    pretrained = torch.zeros(max(word_to_id.values()) + 1, EMB_DIM)
    scale = torch.sqrt(torch.Tensor([3.0 / EMB_DIM]))[0]
    for word, i in word_to_id.items():
        if word in GloVe:
            pretrained[i] = torch.FloatTensor(GloVe[word])
        else:
            vect = torch.FloatTensor(EMB_DIM).uniform_(-scale, scale)
            pretrained[i] = torch.FloatTensor(vect)

    return pretrained

In [44]:
# Takes about 15 seconds to run
start = time.time()
pretrained = init_glove(GloVe)
print(time.time() - start)

15.289496183395386


In [45]:
pretrained.shape

torch.Size([1085161, 300])